# ESM計算を行う

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import copy
import importlib
import requests

## 準備

Google Driveをマウントする。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

QEのファイル一式を取ってくる。

In [ ]:
if not os.path.exists("/content/q-e/"):
    %cd /content/
    !unzip /content/drive/MyDrive/q-e.zip

パスを設定

In [ ]:
os.environ['PATH'] = "/content/q-e/bin:"+os.environ['PATH']
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun --allow-run-as-root -np 4 pw.x -in PREFIX.pwi > PREFIX.pwo"
qef_pseudo = 'http://www.quantum-espresso.org/wp-content/uploads/upf_files/'
pseudo_dir = "/content/q-e/pseudo/"

擬ポテンシャルがなければを取ってくる。

In [ ]:
pseudopotentials = {'Al':'Al.pbe-n-van.UPF'}
for v in pseudopotentials.values():
    local_file = pseudo_dir + v
    if not os.path.isfile(local_file):
        remote_url = qef_pseudo + v
        res = requests.get(remote_url)
        try:
            res.raise_for_status()
        except Exception as exc:
            print('Something went wrong: {}'.format(exc))
        play_file = open(local_file, 'wb')
        for chunk in res.iter_content(100000):
            play_file.write(chunk)

## Al(100)のスラブモデル

ASEを使ってスラブモデルを作成する。

In [ ]:
lvac = 8.0
rvac = 10.0
vac = (lvac + rvac)/2.0
slab = fcc100('Al', size=(1,1,2), vacuum=vac)
slab.wrap() # wrap extended atoms into a unit cell.
slab.translate((0.0,0.0,(lvac-rvac)/2.0))
fig, ax = plt.subplots(1,2, figsize=(12, 6))
ax[0].set_axis_off()
ax[1].set_axis_off()
plot_atoms(slab, ax[0], radii=1.0, rotation=('0x,0y,0z'))
plot_atoms(slab, ax[1], radii=1.0, rotation=('90x,90y,90z'))
plt.show()

ESM計算用にスラブの位置をずらす

In [ ]:
slab_ESM = copy.deepcopy(slab)
slab_ESM.translate((0.0,0.0,-slab.cell[2,2]/2.0)) # shift atoms to fit ESM/ESM-RISM model

スラブが意図した通り作成できているかを```slab_ESM```の中身を見て確認する。

In [ ]:
aobj = slab_ESM
print(f' Unit cell: a = ({aobj.cell[0,0]:9.5f}, {aobj.cell[0,1]:9.5f}, {aobj.cell[0,2]:9.5f})')
print(f'            b = ({aobj.cell[1,0]:9.5f}, {aobj.cell[1,1]:9.5f}, {aobj.cell[1,2]:9.5f})')
print(f'            c = ({aobj.cell[2,0]:9.5f}, {aobj.cell[2,1]:9.5f}, {aobj.cell[2,2]:9.5f})')
print(f' Number of atoms: {len(aobj.positions):5d}')
print(f' Species, Positions:')
for i in range(len(aobj.positions)):
    print(f'  \'{aobj.symbols[i]:<2}\' ({aobj.positions[i,0]:9.5f}, {aobj.positions[i,1]:9.5f}, {aobj.positions[i,2]:9.5f})')

計算を実行する。

In [ ]:
input_data = {
    'control': {
        'calculation': 'scf',
        'restart_mode': 'from_scratch',
        'prefix': 'Al100_bc2',
        'disk_io': 'low'
    },
    'system': {
        'ibrav': 0,
        'ecutwfc': 20,
        'ecutrho': 160,
        'occupations' : 'smearing',
        'smearing':'mp',
        'degauss' : 0.03,
        'assume_isolated': 'esm',
        'esm_bc': 'bc1'
    },
    'electrons': {
        'mixing_beta': 0.3
    }
}

calc = Espresso(pseudo_dir=pseudo_dir,
                pseudopotentials=pseudopotentials,
                kpts=(16, 16, 1),
                koffset=(1, 1, 0),
                input_data=input_data)
slab_ESM.set_calculator(calc)

slab_ESM.get_potential_energy()
fermi_level = calc.get_fermi_level()
print(fermi_level)